# 動態網頁爬蟲 - 使用 Selenium

* 了解 Selenium 用於動態網頁爬蟲的原理
* 能夠使用 Selenium 撰寫動態網頁爬蟲

## 作業目標


* 根據範例使用 selenuim 取出 高雄市小港區 2019 年 空氣品質 資料，並用一個適合的資料結構儲存
* Hint: 將所有資料存在一個變數當中



In [19]:
from selenium.webdriver.chrome import service
from selenium import webdriver
from selenium.webdriver.support.ui import Select
from bs4 import BeautifulSoup

# using opera driver
webDriverServer = service.Service('./Data/operadriver_win64/operadriver.exe')
webDriverServer.start()

capabilities = {
    'operaOptions': {
        'binary': 'C:/Program Files (x86)/Opera/launcher.exe'
    }
}

browser = webdriver.Remote(webDriverServer.service_url, webdriver.DesiredCapabilities.OPERA)

browser.get("http://taqm.epa.gov.tw/taqm/tw/MonthlyAverage.aspx")
#browser.get('https://www.google.com/')

selectSite = Select(browser.find_element_by_id("ctl05_ddlSite"))
selectSite.select_by_value("58")
selectYear = Select(browser.find_element_by_id("ctl05_ddlYear"))
selectYear.select_by_value("2019")

browser.find_element_by_id('ctl05_btnQuery').click()

![](https://imgur.com/LwGGT8c.jpg)

In [20]:
# 取得資料（等瀏覽器資料出現後才執行）
html_source = browser.page_source
html_source

'<html xmlns="http://www.w3.org/1999/xhtml" lang="zh-TW"><head id="Head1"><base href="https://taqm.epa.gov.tw/taqm/tw/"><meta http-equiv="X-UA-Compatible" content="IE=edge"><meta http-equiv="Content-Type" content="text/html; charset=UTF-8"><title>行政院環境保護署－空氣品質監測網 - 月平均值查詢</title><meta name="title" content="空氣品質監測網(TAQM)">\n<!--link rel="govtwsimpledc.xsd" href="http://www.gov.tw/schema/dc/" title="Dublin Core" /--> \n<link type="text/css" href="/taqm/tw/style/all.min.css?v=20180508c" rel="stylesheet">\n<script type="text/javascript" async="" src="https://www.google-analytics.com/analytics.js"></script><script type="text/javascript" src="/taqm/script/jquery.min.js"></script>\n<script type="text/javascript" src="/taqm/script/jquery.myslide2.min.js"></script>\n<script type="text/javascript">\nfunction DownloadReport(url) {\n\tvar win = window.open("DownloadReport.aspx?file="+escape(url),"download" /*,"modal=yes,menubar=no,scrollbars=no,status=no,titlebar=no,toolbar=no,height=200px,width=3

In [69]:
soup = BeautifulSoup(html_source, 'html5lib')
table = soup.find('table', class_='TABLE_G')
print(table.prettify())

<table align="Center" border="1" cellpadding="3" cellspacing="0" class="TABLE_G" id="ctl05_gv" rules="all" style="border-color:Black;border-width:1px;border-style:None;border-collapse:collapse;">
 <tbody>
  <tr style="color:Black;font-weight:normal;">
   <th scope="col">
    監測項目
   </th>
   <th scope="col">
    單位
   </th>
   <th scope="col">
    監測日期
   </th>
   <th scope="col">
    監測值
   </th>
   <th scope="col">
    標註
   </th>
  </tr>
  <tr style="color:Black;">
   <td class="no-alt" rowspan="12" style="white-space:nowrap;" valign="top">
    SO2
   </td>
   <td class="no-alt" rowspan="12" valign="top">
    ppb
   </td>
   <td>
    2019/01
   </td>
   <td>
    4.10
   </td>
   <td>
   </td>
  </tr>
  <tr class="ALT" style="color:Black;">
   <td>
    2019/02
   </td>
   <td>
    3.30
   </td>
   <td>
   </td>
  </tr>
  <tr style="color:Black;">
   <td>
    2019/03
   </td>
   <td>
    3.90
   </td>
   <td>
   </td>
  </tr>
  <tr class="ALT" style="color:Black;">
   <td>
    2019/04

In [86]:
import re

chemicals_dates_values_list = dict()
dates_values_list = dict()

dates_list = table.find_all("td", string = re.compile("2019/"))
measured_value_list = []
for date in dates_list:
    measured_value = date.next_sibling
    measured_value_list.append(measured_value)

print(measured_value_list)
chemicals_list = table.find_all("td", style = "white-space:nowrap;")
n_row = 0
for chemical in chemicals_list:
    
    for index in range( int(chemical["rowspan"]) ):
        dates_values_list[ dates_list[n_row].string ] = measured_value_list[n_row].string
        n_row += 1
    chemicals_dates_values_list[chemical.string] = dates_values_list.copy()
    
print(chemicals_dates_values_list)

[<td>4.10</td>, <td>3.30</td>, <td>3.90</td>, <td>3.40</td>, <td>3.30</td>, <td>4.10</td>, <td>4</td>, <td>4.60</td>, <td>3.50</td>, <td>3.20</td>, <td>3.60</td>, <td>3</td>, <td>0.67</td>, <td>0.48</td>, <td>0.50</td>, <td>0.39</td>, <td>0.38</td>, <td>0.24</td>, <td>0.28</td>, <td>0.33</td>, <td>0.32</td>, <td>0.36</td>, <td>0.46</td>, <td>0.54</td>, <td>24.70</td>, <td>29.40</td>, <td>29.20</td>, <td>30.40</td>, <td>28.60</td>, <td>16.30</td>, <td>15.50</td>, <td>19.70</td>, <td>35.70</td>, <td>44</td>, <td>34.80</td>, <td>22.30</td>, <td>78</td>, <td>57</td>, <td>52</td>, <td>42</td>, <td>35</td>, <td>20</td>, <td>24</td>, <td>27</td>, <td>35</td>, <td>50</td>, <td>63</td>, <td>54</td>, <td>38.01</td>, <td>25.62</td>, <td>27.84</td>, <td>21.83</td>, <td>22.55</td>, <td>17.15</td>, <td>17.71</td>, <td>19.66</td>, <td>18.66</td>, <td>19.50</td>, <td>28.99</td>, <td>33.84</td>, <td>7.95</td>, <td>4.76</td>, <td>5.41</td>, <td>4.16</td>, <td>4.58</td>, <td>5.08</td>, <td>5.16</td>, <td

In [87]:
import pandas as pd
pd.DataFrame(chemicals_dates_values_list)

,SO2,CO,O3,PM10,NOx,NO,NO2,THC,NMHC,CH4
2019/01,4.10,0.67,24.70,78,38.01,7.95,30.06,2.27,0.27,2
2019/02,3.30,0.48,29.40,57,25.62,4.76,20.86,2.11,0.17,1.94
2019/03,3.90,0.50,29.20,52,27.84,5.41,22.43,2.12,0.18,1.94
2019/04,3.40,0.39,30.40,42,21.83,4.16,17.68,2.02,0.14,1.88
2019/05,3.30,0.38,28.60,35,22.55,4.58,17.98,2.02,0.16,1.86
2019/06,4.10,0.24,16.30,20,17.15,5.08,12.08,1.92,0.13,1.79
2019/07,4,0.28,15.50,24,17.71,5.16,12.54,1.93,0.14,1.80
2019/08,4.60,0.33,19.70,27,19.66,5.17,14.49,1.97,0.15,1.82
2019/09,3.50,0.32,35.70,35,18.66,3.63,15.03,1.98,0.13,1.86
2019/10,3.20,0.36,44,50,19.50,3.50,15.99,2.05,0.13,1.92
